# Coursera- IBM Applied Data Science Capstone

## 1. Import libraries¶

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import geopy

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
df = pd.read_csv('IN.csv')
df.head(11)

,Country,Pincode,Placename,State,Column5,District,Column7,Taluka,Column9,Latitude,Longitude,Column12
0,IN,744101,Marine Jetty,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.6667,92.7500,3
1,IN,744101,Port Blair,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.6667,92.7500,4
2,IN,744101,N.S.Building,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.6667,92.7500,3
3,IN,744102,Haddo,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.6833,92.7167,4
4,IN,744102,Chatham,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.7000,92.6667,3
5,IN,744102,Herbertabad,Andaman & Nicobar Islands,1,South Andaman,NaN,Ferrargunj,NaN,11.7167,92.6167,4
6,IN,744102,Delanipur,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.7000,92.6667,3
7,IN,744102,Radio Colony,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.7000,92.6667,3
8,IN,744103,Minnie Bay,Andaman & Nicobar Islands,1,South Andaman,NaN,Portblair,NaN,11.6651,92.7121,1
9,IN,744103,Brijgunj,Andaman & Nicobar Islands,1,South Andaman,NaN,Port Blair,NaN,11.6651,92.7121,1


In [3]:
df.shape

(154809, 12)

Wohh. SInce it is data of whole country it's huge in volume. But we only want to focus on Mumbai so let's extract rows in which district is Mumbai

In [4]:
mdf = df[df['District'] == "Mumbai"]
mdf.head(11)

,Country,Pincode,Placename,State,Column5,District,Column7,Taluka,Column9,Latitude,Longitude,Column12
36230,IN,400001,Mumbai G.P.O.,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36231,IN,400001,Bazargate,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36232,IN,400001,Town Hall (Mumbai),Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36233,IN,400001,Tajmahal,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36235,IN,400001,Stock Exchange,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36236,IN,400001,M.P.T.,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,18.6291,72.8919,1
36237,IN,400002,Kalbadevi,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,17.0509,73.2910,4
36238,IN,400002,S. C. Court,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,17.0509,73.2910,3
36239,IN,400002,Ramwadi,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,17.0509,73.2910,3
36240,IN,400002,Thakurdwar,Maharashtra,16,Mumbai,NaN,Mumbai,NaN,17.0509,73.2910,3


In [5]:
mdf.shape

(239, 12)

Okay. Now let's only keep the columns we want for further.

In [6]:
mdf = mdf[['Pincode','Placename','District','State','Taluka','Latitude','Longitude']]
mdf.head(11)

,Pincode,Placename,District,State,Taluka,Latitude,Longitude
36230,400001,Mumbai G.P.O.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36231,400001,Bazargate,Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36232,400001,Town Hall (Mumbai),Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36233,400001,Tajmahal,Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36235,400001,Stock Exchange,Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36236,400001,M.P.T.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919
36237,400002,Kalbadevi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910
36238,400002,S. C. Court,Mumbai,Maharashtra,Mumbai,17.0509,73.2910
36239,400002,Ramwadi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910
36240,400002,Thakurdwar,Mumbai,Maharashtra,Mumbai,17.0509,73.2910


In [7]:
mdf = mdf.dropna()
mdf.shape

(181, 7)

In [8]:
mdf['Full_address'] = mdf['Placename'] + "," + mdf['Taluka'] + "," + mdf["District"] 
mdf.head(11)

,Pincode,Placename,District,State,Taluka,Latitude,Longitude,Full_address
36230,400001,Mumbai G.P.O.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Mumbai G.P.O.,Mumbai,Mumbai"
36231,400001,Bazargate,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Bazargate,Mumbai,Mumbai"
36232,400001,Town Hall (Mumbai),Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Town Hall (Mumbai),Mumbai,Mumbai"
36233,400001,Tajmahal,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Tajmahal,Mumbai,Mumbai"
36235,400001,Stock Exchange,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Stock Exchange,Mumbai,Mumbai"
36236,400001,M.P.T.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"M.P.T.,Mumbai,Mumbai"
36237,400002,Kalbadevi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Kalbadevi,Mumbai,Mumbai"
36238,400002,S. C. Court,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"S. C. Court,Mumbai,Mumbai"
36239,400002,Ramwadi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Ramwadi,Mumbai,Mumbai"
36240,400002,Thakurdwar,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Thakurdwar,Mumbai,Mumbai"


Now Finally! Our dataframe is ready to be used. But first save our cleaned dataframe as .csv file also so it can be used in future or can be shared directly. 

In [10]:
mdf.to_csv('mumbailatlong.csv')

## Define Foursquare credentials and version

In [11]:
CLIENT_ID = 'YOAFQIVX0YUR4TCG25GNWVBSPYN5YAVN0JSRXB2UV4LLLKHK'
CLIENT_SECRET = '452ZMKLVKSBDM5OKJHBR2EW54PU3WXDPRKPFGFJUPJ0CXWAQ'
ACCESS_TOKEN = '2Y2IB4IL2V5BAVXMXQNODMXG3BKUUMPOZTZKF3UWM2HNKTJ0'
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YOAFQIVX0YUR4TCG25GNWVBSPYN5YAVN0JSRXB2UV4LLLKHK
CLIENT_SECRET:452ZMKLVKSBDM5OKJHBR2EW54PU3WXDPRKPFGFJUPJ0CXWAQ


In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1111
    LIMIT=111
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
mumbai_venues = getNearbyVenues(names=mdf['Full_address'],
                                   latitudes=mdf['Latitude'],
                                   longitudes=mdf['Longitude']
                                  )

Mumbai G.P.O.,Mumbai,Mumbai
Bazargate,Mumbai,Mumbai
Town Hall (Mumbai),Mumbai,Mumbai
Tajmahal,Mumbai,Mumbai
Stock Exchange,Mumbai,Mumbai
M.P.T.,Mumbai,Mumbai
Kalbadevi,Mumbai,Mumbai
S. C. Court,Mumbai,Mumbai
Ramwadi,Mumbai,Mumbai
Thakurdwar,Mumbai,Mumbai
Girgaon,Mumbai,Mumbai
Opera House,Mumbai,Mumbai
Ambewadi (Mumbai),Mumbai,Mumbai
Charni Road,Mumbai,Mumbai
Chaupati,Mumbai,Mumbai
Madhavbaug,Mumbai,Mumbai
Colaba,Mumbai,Mumbai
Holiday Camp,Mumbai,Mumbai
V.W.T.C.,Mumbai,Mumbai
Asvini,Mumbai,Mumbai
Malabar Hill,Mumbai,Mumbai
Tardeo,Mumbai,Mumbai
Grant Road,Mumbai,Mumbai
S V Marg,Mumbai,Mumbai
N.S.Patkar Marg,Mumbai,Mumbai
Bharat Nagar (Mumbai),Mumbai,Mumbai
Falkland Road,Mumbai,Mumbai
Mumbai Central,Mumbai,Mumbai
M A Marg,Mumbai,Mumbai
J.J.Hospital,Mumbai,Mumbai
Kamathipura,Mumbai,Mumbai
Princess Dock,Mumbai,Mumbai
Mazgaon,Mumbai,Mumbai
Mazgaon Road,Mumbai,Mumbai
V K Bhavan,Mumbai,Mumbai
Mazgaon Dock,Mumbai,Mumbai
Dockyard Road,Mumbai,Mumbai
Agripada,Mumbai,Mumbai
Chinchpokli,Mumbai,Mumba

In [13]:
address = 'Mumbai, Maharastra'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai City are 19.0759899, 72.8773928.


In [14]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, taluka, placename in zip(mdf['Latitude'], mdf['Longitude'], mdf['Taluka'], mdf['Placename']):
    label = '{}, {}'.format(placename, taluka)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

However, for illustration purposes, let's simplify the above map and segment and cluster only the places in Borivali East. So let's slice the original dataframe and create a new dataframe of the Borivali East data.

In [15]:
borivali_east = mdf[mdf['Taluka'] == "Borivali East"]
borivali_east.head()

,Pincode,Placename,District,State,Taluka,Latitude,Longitude,Full_address
36390,400066,Rajendra Nagar (Mumbai),Mumbai,Maharashtra,Borivali East,19.1699,72.8504,"Rajendra Nagar (Mumbai),Borivali East,Mumbai"
36392,400066,Magthane,Mumbai,Maharashtra,Borivali East,19.1699,72.8504,"Magthane,Borivali East,Mumbai"
36393,400066,Daulat Nagar (Mumbai),Mumbai,Maharashtra,Borivali East,19.1699,72.8504,"Daulat Nagar (Mumbai),Borivali East,Mumbai"
36394,400066,S. K.Nagar,Mumbai,Maharashtra,Borivali East,19.1699,72.8504,"S. K.Nagar,Borivali East,Mumbai"


Let's use geographical coordinates of **Borivali East**

In [16]:
address = 'Borivali East, Mumbai'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Borivali East are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Borivali East are 19.2267228, 72.8619328.


As we did with whole Mumbai city. This time let's visulize places in Borivali East with folium library.

In [17]:
map_BorivaliEast = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, taluka, placename in zip(borivali_east['Latitude'], borivali_east['Longitude'], borivali_east['Taluka'], borivali_east['Placename']):
    label = '{}, {}'.format(placename, taluka)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_BorivaliEast)  
    
map_BorivaliEast

Unfortunately, Maybe due to not so precise latitudes and longitudes of our data we are only can only see one dot that is of the last entry in our new dataframe and it is S. K.Nagar,Borivali East,Mumbai. You can see the name by clicking on the blue dots.

Similarly lets find and visulize places for Taluka Mumbai.

In [18]:
mumbai_city = mdf[mdf['Taluka'] == "Mumbai"]
mumbai_city

,Pincode,Placename,District,State,Taluka,Latitude,Longitude,Full_address
36230,400001,Mumbai G.P.O.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Mumbai G.P.O.,Mumbai,Mumbai"
36231,400001,Bazargate,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Bazargate,Mumbai,Mumbai"
36232,400001,Town Hall (Mumbai),Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Town Hall (Mumbai),Mumbai,Mumbai"
36233,400001,Tajmahal,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Tajmahal,Mumbai,Mumbai"
36235,400001,Stock Exchange,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Stock Exchange,Mumbai,Mumbai"
36236,400001,M.P.T.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"M.P.T.,Mumbai,Mumbai"
36237,400002,Kalbadevi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Kalbadevi,Mumbai,Mumbai"
36238,400002,S. C. Court,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"S. C. Court,Mumbai,Mumbai"
36239,400002,Ramwadi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Ramwadi,Mumbai,Mumbai"
36240,400002,Thakurdwar,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Thakurdwar,Mumbai,Mumbai"


In [19]:
address = 'Mumbai,Mumbai'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai,Mumbai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai,Mumbai are 18.94136785, 72.83526733695547.


In [20]:
map_mumbaicity = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, taluka, placename in zip(mumbai_city['Latitude'], mumbai_city['Longitude'], mumbai_city['Taluka'], mumbai_city['Placename']):
    label = '{}, {}'.format(placename, taluka)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbaicity)  
    
map_mumbaicity

#### Let's explore the first placename in our dataframe.

In [26]:

mumbai_city.loc[36230,'Placename'] #it was tricky one! Since indexing starts from 36230 in our mumbai_city dataframe we had to start form there.

'Mumbai G.P.O.'

### Get the Placename's latitude and longitude values.

In [28]:
placename_latitude = mumbai_city.loc[36230, 'Latitude'] # neighborhood latitude value
placename_longitude = mumbai_city.loc[36230, 'Longitude'] # neighborhood longitude value
placename = mumbai_city.loc[36230, 'Placename'] # place name

print('Latitude and longitude values of {} are {}, {}.'.format(placename, 
                                                               placename_latitude, 
                                                               placename_longitude))

Latitude and longitude values of Mumbai G.P.O. are 18.6291, 72.8919.


## Now, let's get the top 100 venues that are in Mumbai G.P.O. within a radius of 500 meters.

In [39]:
LIMIT = 111 # limit of number of venues returned by Foursquare API
radius = 11111 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    placename_latitude, 
    placename_longitude, 
    radius, 
    LIMIT)
url #display url

'https://api.foursquare.com/v2/venues/explore?&client_id=YOAFQIVX0YUR4TCG25GNWVBSPYN5YAVN0JSRXB2UV4LLLKHK&client_secret=452ZMKLVKSBDM5OKJHBR2EW54PU3WXDPRKPFGFJUPJ0CXWAQ&v=20180604&ll=18.6291,72.8919&radius=11111&limit=111'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604b6decd34f6c62afda6d07'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Alībāg',
  'headerFullLocation': 'Alībāg',
  'headerLocationGranularity': 'city',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 18.7290990999991,
    'lng': 72.99723107595001},
   'sw': {'lat': 18.529100900000902, 'lng': 72.78656892405}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4df64e1da809141629abb9f6',
       'name': 'Radisson Blu',
       'location': {'address': 'Gondhalpada',
        'lat': 18.665312358186252,
        'lng': 72.88922443985939,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18.665312358186252,
          'lng': 72.

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [44]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-44-4243e3e1ab9c>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Radisson Blu,Motel,18.665312,72.889224
1,Nagaon Beach,Beach,18.609048,72.901019
2,Alibag Beach,Beach,18.639854,72.872032
3,Sanman Restaurant,Indian Restaurant,18.643646,72.875801
4,Alibag,City,18.655768,72.878470
5,Fulora ( फ़ुलोरा ),Indian Restaurant,18.647190,72.874170
6,Kashid Beach,Beach,18.616678,72.902236
7,Smile Fresh Cafe,Café,18.646723,72.873670
8,Hotel Ramakanth,Indian Restaurant,18.648457,72.874414
9,Aparanta Restaurant,Indian Restaurant,18.664879,72.889927


Let's find number of venues returned by Foursquare near Mumbai G.P.O.

In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


## Let's create a function to repeat the same process to all the place names in Mumbai

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [56]:
mumbai_venues = getNearbyVenues(names=mumbai_city['Placename'],
                                   latitudes=mumbai_city['Latitude'],
                                   longitudes=mumbai_city['Longitude']
                                  )

Mumbai G.P.O.
Bazargate
Town Hall (Mumbai)
Tajmahal
Stock Exchange
M.P.T.
Kalbadevi
S. C. Court
Ramwadi
Thakurdwar
Girgaon
Opera House
Ambewadi (Mumbai)
Charni Road
Chaupati
Madhavbaug
Colaba
Holiday Camp
V.W.T.C.
Asvini
Malabar Hill
Tardeo
Grant Road
S V Marg
N.S.Patkar Marg
Bharat Nagar (Mumbai)
Falkland Road
Mumbai Central
M A Marg
J.J.Hospital
Kamathipura
Princess Dock
Mazgaon
Mazgaon Road
V K Bhavan
Mazgaon Dock
Dockyard Road
Agripada
Chinchpokli
Jacob Circle
Haines Road
Parel
Chamarbaug
Haffkin Institute
Lal Baug
Parel Naka
Delisle Road
Naigaon (Mumbai)
Dadar
Dadar Colony
Sewri
Mahim
Mahim Bazar
Mori Road
Kapad Bazar
Mahim East
Dharavi Road
Dharavi
Worli
Worli Naka
Matunga
Marine Lines
Churchgate
Central Building
Nariman Point
New Yogakshema
Sion
Chunabhatti
Nehru Nagar (Mumbai)
Prabhadevi
New Prabhadevi Road
Cumballa Sea Face
Gowalia Tank
Dr Deshmukh Marg
Cumballa Hill
V J B Udyan
Ranade Road
Gokhale Road (Mumbai)
Bhawani Shankar
Bhawani Shankar Rd
S V S Marg
Shivaji Park (Mumba

Let's check the size of the resulting dataframe

In [60]:
print(mumbai_venues.shape)
mumbai_venues.head()

(1328, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mumbai G.P.O.,18.6291,72.8919,Washing Machine Repair,18.628739,72.891101,Other Repair Shop
1,Mumbai G.P.O.,18.6291,72.8919,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service
2,Mumbai G.P.O.,18.6291,72.8919,Kadam Holidays,18.629150,72.888194,Hotel
3,Bazargate,18.6291,72.8919,Washing Machine Repair,18.628739,72.891101,Other Repair Shop
4,Bazargate,18.6291,72.8919,ACHI BIZ SERVICES PTE. LTD.,18.630000,72.891530,Business Service


In [59]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A I Staff Colony,10,10,10,10,10,10
Agripada,4,4,4,4,4,4
Airport (Mumbai),5,5,5,5,5,5
Ambewadi (Mumbai),19,19,19,19,19,19
Andheri,6,6,6,6,6,6
Andheri East,7,7,7,7,7,7
Andheri Railway Station,2,2,2,2,2,2
Antop Hill,16,16,16,16,16,16
Asvini,7,7,7,7,7,7


Let's find out how many unique categories can be curated from all the returned venues

In [65]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 104 uniques categories.


##  Analyzing Each Neighborhood

In [61]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
manhattan_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Accessories Store,Aquarium,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bus Line,Bus Station,Business Service,Café,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Venue,Nightclub,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Pub,Racetrack,Recreation Center,Restaurant,Roof Deck,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai G.P.O.
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai G.P.O.
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Mumbai G.P.O.
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bazargate
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bazargate


let's examine the new dataframe size

In [62]:
mumbai_onehot.shape

(1328, 105)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [63]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped

,Neighborhood,Accessories Store,Aquarium,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beach,Bengali Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Breakfast Spot,Brewery,Bus Line,Bus Station,Business Service,Café,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lounge,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Music Venue,Nightclub,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Pool,Pub,Racetrack,Recreation Center,Restaurant,Roof Deck,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Smoke Shop,Snack Place,Spa,Spanish Restaurant,Steakhouse,Supermarket,Surf Spot,Tea Room,Tennis Court,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Women's Store,Yoga Studio
0,A I Staff Colony,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,Agripada,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.250000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00000,0.000000,0.000000
2,Airport (Mumbai),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Let's confirm the new size

In [64]:
mumbai_grouped.shape

(133, 105)

Let's print each neighborhood along with the top 5 most common venues

In [66]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A I Staff Colony----
               venue  freq
0           Tea Room   0.2
1  Indian Restaurant   0.1
2          Racetrack   0.1
3        Coffee Shop   0.1
4               Café   0.1


----Agripada----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.25
1                 History Museum  0.25
2           Fast Food Restaurant  0.25
3                      Multiplex  0.25
4              Accessories Store  0.00


----Airport (Mumbai)----
                venue  freq
0  Italian Restaurant   0.2
1                Park   0.2
2                 Gym   0.2
3                Café   0.2
4        Dessert Shop   0.2


----Ambewadi (Mumbai)----
                  venue  freq
0     Indian Restaurant  0.11
1       Harbor / Marina  0.11
2                  Café  0.11
3  Fast Food Restaurant  0.05
4             Juice Bar  0.05


----Andheri----
                venue  freq
0         Bus Station  0.17
1  Athletics & Sports  0.17
2              Bakery  0.17
3         Pizza Place  0.17

**Let's put that into a pandas dataframe**

First, let's write a function to sort the venues in descending order.

In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A I Staff Colony,Tea Room,Indian Restaurant,Racetrack,Coffee Shop,Café,Spa,Modern European Restaurant,Furniture / Home Store,Asian Restaurant,Mexican Restaurant
1,Agripada,Vegetarian / Vegan Restaurant,History Museum,Fast Food Restaurant,Multiplex,Accessories Store,Movie Theater,Pizza Place,Pharmacy,Performing Arts Venue,Park
2,Airport (Mumbai),Italian Restaurant,Park,Gym,Café,Dessert Shop,Music Venue,Pool,Plaza,Pizza Place,Pharmacy
3,Ambewadi (Mumbai),Indian Restaurant,Harbor / Marina,Café,Fast Food Restaurant,Juice Bar,Restaurant,Food Truck,Smoke Shop,Grocery Store,Gym
4,Andheri,Bus Station,Athletics & Sports,Bakery,Pizza Place,Gym,Bike Rental / Bike Share,Other Repair Shop,Music Venue,Nightclub,Other Great Outdoors


# Clustering Neighborhoods with k-means algorithm

In [90]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 4, 2, 4, 3, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [92]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head(11)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,A I Staff Colony,Tea Room,Indian Restaurant,Racetrack,Coffee Shop,Café,Spa,Modern European Restaurant,Furniture / Home Store,Asian Restaurant,Mexican Restaurant
1,0,Agripada,Vegetarian / Vegan Restaurant,History Museum,Fast Food Restaurant,Multiplex,Accessories Store,Movie Theater,Pizza Place,Pharmacy,Performing Arts Venue,Park
2,0,Airport (Mumbai),Italian Restaurant,Park,Gym,Café,Dessert Shop,Music Venue,Pool,Plaza,Pizza Place,Pharmacy
3,0,Ambewadi (Mumbai),Indian Restaurant,Harbor / Marina,Café,Fast Food Restaurant,Juice Bar,Restaurant,Food Truck,Smoke Shop,Grocery Store,Gym
4,0,Andheri,Bus Station,Athletics & Sports,Bakery,Pizza Place,Gym,Bike Rental / Bike Share,Other Repair Shop,Music Venue,Nightclub,Other Great Outdoors
5,0,Andheri East,Diner,Hotel,Asian Restaurant,Hotel Bar,Vegetarian / Vegan Restaurant,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue
6,2,Andheri Railway Station,Business Service,Accessories Store,Pub,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park,Other Repair Shop,Other Great Outdoors
7,0,Antop Hill,Coffee Shop,Indian Restaurant,Café,Bus Station,Gym,Farmers Market,Snack Place,Pub,Juice Bar,Lounge
8,0,Asvini,Indian Restaurant,Jewelry Store,Restaurant,Pharmacy,Garden,Boutique,Café,Music Venue,Plaza,Pizza Place
9,0,Azad Nagar (Mumbai),Bus Station,Athletics & Sports,Bakery,Pizza Place,Gym,Bike Rental / Bike Share,Other Repair Shop,Music Venue,Nightclub,Other Great Outdoors


In [102]:
mumbai_merged = mumbai_city

# merge mumbai_grouped with mumbai_city to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Placename')
#mumbai_merged['Cluster Labels'].astype(int)

mumbai_merged # check the last columns!

,Pincode,Placename,District,State,Taluka,Latitude,Longitude,Full_address,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36230,400001,Mumbai G.P.O.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Mumbai G.P.O.,Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36231,400001,Bazargate,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Bazargate,Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36232,400001,Town Hall (Mumbai),Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Town Hall (Mumbai),Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36233,400001,Tajmahal,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Tajmahal,Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36235,400001,Stock Exchange,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"Stock Exchange,Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36236,400001,M.P.T.,Mumbai,Maharashtra,Mumbai,18.6291,72.8919,"M.P.T.,Mumbai,Mumbai",3.0,Hotel,Other Repair Shop,Business Service,Accessories Store,Movie Theater,Plaza,Pizza Place,Pharmacy,Performing Arts Venue,Park
36237,400002,Kalbadevi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Kalbadevi,Mumbai,Mumbai",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36238,400002,S. C. Court,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"S. C. Court,Mumbai,Mumbai",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36239,400002,Ramwadi,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Ramwadi,Mumbai,Mumbai",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36240,400002,Thakurdwar,Mumbai,Maharashtra,Mumbai,17.0509,73.2910,"Thakurdwar,Mumbai,Mumbai",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finally, let's visualize the resulting clusters

In [98]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Placename'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Acknowledgement

- In this project, I have gone through the process of identifying the business problems, extracting and preparing the data, visualizing the results, performing machine learning by clustering the data into clusters based on their frequency similarities, tackling and reaching to a definitive solution to business problems.
- Data used was downloaded from "www.geonames.org" and converted to comma seperated value(CSV) file and then used in this project.